 # Opis notatnika

 Ten notatnik inicjuje naszą pracę nad warsztatem końcowym. Naszym zadaniem tutaj jest pobranie udostępnionych nam danych do obszaru roboczego, które w następnym kroku wgramy na naszą bazę danych. Ich obróbka oraz analiza zostanie przeprowadzona w specjalnie do tego celu przygotowanych kolejnych notatnikach.

 Na potrzeby tego warsztatu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.

 Przy wykonywaniu tego zadania możesz posłużyć się tym tokenem: `WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr`.

 ### Uwagi
 - Ze względów ćwiczeniowych, konstrukcja poszczególnych endpointów jest różna – w trakcie pracy dokładnie przyjrzyj się, w jaki sposób należy wykonać zapytanie, aby otrzymać odpowiedź.
 - Pamiętaj o dodaniu `sleep` pomiędzy poszczególnymi wywołaniami endpoint.
 - Limit wywołań API to 500/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

 Tutaj zaimportuj wymagane biblioteki

In [1]:
import requests
import pandas as pd 
import time
import json
from time import sleep
from dateutil.relativedelta import relativedelta
from datetime import datetime
import concurrent.futures

 Tutaj zdefiniuj parametry połączenia do API

In [2]:
base_url = "https://api-datalab.coderslab.com/api/" 
headers = { "Authorization": "WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr" }

Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [3]:
airport_df = pd.read_csv("..\\data\\airports.csv")

 # Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.

 ### Wskazówki
 - Nie wszystkie lotniska dostępne w pliku `airports.csv`, są dostępne w endpoint. Zadbaj o odpowiednie obsłużenie takiej sytuacji,
 - Do skonwertowania wyników przydatna może okazać się metoda `Pandas` - [from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html),
 - Artykuł LMS: `Python - analiza danych > Dzień 4 - API > Uwierzytelnianie`
 - Artykuł LMS: `Python - analiza danych > Przygotowanie do zjazdu 2`

 Tutaj pobierz dane z endpoint'u `airport`

In [ ]:
url = "https://api-datalab.coderslab.com/api/airport"

airport_data = []

for index, row in airport_df.iterrows():
    airport_id = row["origin_airport_id"]
    api_url = f"{url}/{airport_id}"
    response = requests.get(api_url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        airport_data.append([data["ORIGIN_AIRPORT_ID"], data["DISPLAY_AIRPORT_NAME"], data["ORIGIN_CITY_NAME"], data["NAME"]])
    else:
        print(f"Error: Could not retrieve data for airport with ID {airport_id}")
    time.sleep(0.5)
        
airport_df = pd.DataFrame(airport_data, columns=["ORIGIN_AIRPORT_ID", "DISPLAY_AIRPORT_NAME", "ORIGIN_CITY_NAME", "NAME"])

In [166]:
print(airport_df.shape)

(97, 4)


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [167]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`.

In [164]:
airport_df.to_csv("..\\data\\raw\\airport_list.csv", index=False)


 # Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.

 Wskazówki:
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`, czyli stworzenie listy, a następnie przekonwertowanie jej w postać ramki.
 - Data początkowa danych to `2019-01-01`, zaś data końcowa to `2020-03-31`, czyli 15 miesięcy,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.
 - Przy dodawaniu miesięcy do daty może przydać się metoda [relativedelta](https://www.geeksforgeeks.org/python-get-month-from-year-and-weekday/).

In [ ]:
# Define the start and end dates
start_date = datetime(2019, 1, 1)
end_date = datetime(2020, 3, 31)
# Define the URL and headers for the API request
url = "https://api-datalab.coderslab.com/api/airportWeather"
headers = {
    "Authorization": "WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr"
}
# Initialize an empty list to store the weather data
weather_data = []
# Loop through each month between the start and end dates
current_date = start_date
while current_date <= end_date:
    # Format the date string to YYYY-MM
    date_str = current_date.strftime("%Y-%m")
    # Make the API request for this date
    response = requests.get(url, headers=headers, params={"date": date_str})
    # If the request is successful, append the weather data to the list
    if response.status_code == 200:
        weather_data += response.json()
        print(f"Successfully fetched data for {date_str}")
    else:
        print(f"Error fetching data for {date_str}: {response.text}")
    # Increment the current date by 1 month
    current_date += relativedelta(months=1)
# Convert the weather data list to a pandas DataFrame
airport_weather_df = pd.DataFrame.from_records(weather_data)


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [138]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape


Successfully fetched data for 2019-01
Successfully fetched data for 2019-02
Successfully fetched data for 2019-03
Successfully fetched data for 2019-04
Successfully fetched data for 2019-05
Successfully fetched data for 2019-06
Successfully fetched data for 2019-07
Successfully fetched data for 2019-08
Successfully fetched data for 2019-09
Successfully fetched data for 2019-10
Successfully fetched data for 2019-11
Successfully fetched data for 2019-12
Successfully fetched data for 2020-01
Successfully fetched data for 2020-02
Successfully fetched data for 2020-03


 ## Zapis do pliku
 Tutaj zapisz ramkę `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`.

In [139]:
airport_weather_df.to_csv('..\\data\\raw\\airport_weather.csv', index=False)


 # Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft` a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


In [168]:
headers = {'Authorization': 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'}
response = requests.get('https://api-datalab.coderslab.com/api/aircraft', headers=headers)
if response.status_code == 200:
    aircraft_data = response.json()
    aircraft_df = pd.DataFrame(aircraft_data)
    print(aircraft_df.head())
else:
    print('Error:', response.status_code)
time.sleep(2)

   MANUFACTURE_YEAR TAIL_NUM  NUMBER_OF_SEATS
0              1944   N54514              0.0
1              1945   N1651M              0.0
2              1953   N100CE              0.0
3              1953   N141FL              0.0
4              1953   N151FL              0.0


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [171]:
print(aircraft_df.shape)

(7383, 3)


In [169]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`.

In [ ]:
aircraft_df.to_csv('..\\data\\raw\\aircraft.csv', index=False)


 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.

In [14]:
import concurrent.futures
import requests
import json
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime
from time import sleep

endpoint_url = 'https://api-datalab.coderslab.com/api'
token = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'
request_offset = 0.3
authorization = {'authorization': token}

flight_endpoint_url = f"{endpoint_url}/flight"

def get_airport_data(airport_id):
    result = []
    start_date = datetime(2019, 1, 1)
    end_date = datetime.now()
    months_amount = (end_date.year - start_date.year) * 12 + end_date.month - start_date.month + 1

    for month in range(months_amount):
        date = start_date + relativedelta(months=month)
        print(f"\t{date}")

        response = get_airport_day_data(airport_id, date)
        result += response

        sleep(request_offset)

    return result

def get_airport_day_data(airport_id, date):
    payload = {
        'airportId': airport_id,
        'date': date.strftime('%Y-%m')
    }

    r = requests.get(
        flight_endpoint_url,
        headers=authorization,
        params=payload)

    response = json.loads(r.text)
    return response

airports = pd.read_csv(r"..\\data\\airports.csv")


flight = []
airports_amount = len(airports)

for idx, airport in enumerate(airports.to_dict(orient='records')):
    print(f"{airport}: {idx + 1} of {airports_amount}")
    airport_flight = get_airport_data(airport["origin_airport_id"])
    flight += airport_flight
    
flight_df = pd.DataFrame.from_records(flight)
flight_df.head()


{'origin_airport_id': 10874}: 1 of 364
	2019-01-01 00:00:00
	2019-02-01 00:00:00
	2019-03-01 00:00:00
	2019-04-01 00:00:00
	2019-05-01 00:00:00
	2019-06-01 00:00:00
	2019-07-01 00:00:00
	2019-08-01 00:00:00
	2019-09-01 00:00:00
	2019-10-01 00:00:00
	2019-11-01 00:00:00
	2019-12-01 00:00:00
	2020-01-01 00:00:00
	2020-02-01 00:00:00
	2020-03-01 00:00:00
	2020-04-01 00:00:00
	2020-05-01 00:00:00
	2020-06-01 00:00:00
	2020-07-01 00:00:00
	2020-08-01 00:00:00
	2020-09-01 00:00:00
	2020-10-01 00:00:00
	2020-11-01 00:00:00
	2020-12-01 00:00:00
	2021-01-01 00:00:00
	2021-02-01 00:00:00
	2021-03-01 00:00:00
	2021-04-01 00:00:00
	2021-05-01 00:00:00
	2021-06-01 00:00:00
	2021-07-01 00:00:00
	2021-08-01 00:00:00
	2021-09-01 00:00:00
	2021-10-01 00:00:00
	2021-11-01 00:00:00
	2021-12-01 00:00:00
	2022-01-01 00:00:00
	2022-02-01 00:00:00
	2022-03-01 00:00:00
	2022-04-01 00:00:00
	2022-05-01 00:00:00
	2022-06-01 00:00:00
	2022-07-01 00:00:00
	2022-08-01 00:00:00
	2022-09-01 00:00:00
	2022-10-01 00:0

	2021-03-01 00:00:00
	2021-04-01 00:00:00
	2021-05-01 00:00:00
	2021-06-01 00:00:00
	2021-07-01 00:00:00
	2021-08-01 00:00:00
	2021-09-01 00:00:00
	2021-10-01 00:00:00
	2021-11-01 00:00:00
	2021-12-01 00:00:00
	2022-01-01 00:00:00
	2022-02-01 00:00:00
	2022-03-01 00:00:00
	2022-04-01 00:00:00
	2022-05-01 00:00:00
	2022-06-01 00:00:00
	2022-07-01 00:00:00
	2022-08-01 00:00:00
	2022-09-01 00:00:00
	2022-10-01 00:00:00
	2022-11-01 00:00:00
	2022-12-01 00:00:00
	2023-01-01 00:00:00
	2023-02-01 00:00:00
{'origin_airport_id': 12007}: 9 of 364
	2019-01-01 00:00:00
	2019-02-01 00:00:00
	2019-03-01 00:00:00
	2019-04-01 00:00:00
	2019-05-01 00:00:00
	2019-06-01 00:00:00
	2019-07-01 00:00:00
	2019-08-01 00:00:00
	2019-09-01 00:00:00
	2019-10-01 00:00:00
	2019-11-01 00:00:00
	2019-12-01 00:00:00
	2020-01-01 00:00:00
	2020-02-01 00:00:00
	2020-03-01 00:00:00
	2020-04-01 00:00:00
	2020-05-01 00:00:00
	2020-06-01 00:00:00
	2020-07-01 00:00:00
	2020-08-01 00:00:00
	2020-09-01 00:00:00
	2020-10-01 00:0

	2019-02-01 00:00:00
	2019-03-01 00:00:00
	2019-04-01 00:00:00
	2019-05-01 00:00:00
	2019-06-01 00:00:00
	2019-07-01 00:00:00
	2019-08-01 00:00:00
	2019-09-01 00:00:00
	2019-10-01 00:00:00
	2019-11-01 00:00:00
	2019-12-01 00:00:00
	2020-01-01 00:00:00
	2020-02-01 00:00:00
	2020-03-01 00:00:00
	2020-04-01 00:00:00
	2020-05-01 00:00:00
	2020-06-01 00:00:00
	2020-07-01 00:00:00
	2020-08-01 00:00:00
	2020-09-01 00:00:00
	2020-10-01 00:00:00
	2020-11-01 00:00:00
	2020-12-01 00:00:00
	2021-01-01 00:00:00
	2021-02-01 00:00:00
	2021-03-01 00:00:00
	2021-04-01 00:00:00
	2021-05-01 00:00:00
	2021-06-01 00:00:00
	2021-07-01 00:00:00
	2021-08-01 00:00:00
	2021-09-01 00:00:00
	2021-10-01 00:00:00
	2021-11-01 00:00:00
	2021-12-01 00:00:00
	2022-01-01 00:00:00
	2022-02-01 00:00:00
	2022-03-01 00:00:00
	2022-04-01 00:00:00
	2022-05-01 00:00:00
	2022-06-01 00:00:00
	2022-07-01 00:00:00
	2022-08-01 00:00:00
	2022-09-01 00:00:00
	2022-10-01 00:00:00
	2022-11-01 00:00:00
	2022-12-01 00:00:00
	2023-01-01 0

	2021-05-01 00:00:00
	2021-06-01 00:00:00
	2021-07-01 00:00:00
	2021-08-01 00:00:00
	2021-09-01 00:00:00
	2021-10-01 00:00:00
	2021-11-01 00:00:00
	2021-12-01 00:00:00
	2022-01-01 00:00:00
	2022-02-01 00:00:00
	2022-03-01 00:00:00
	2022-04-01 00:00:00
	2022-05-01 00:00:00
	2022-06-01 00:00:00
	2022-07-01 00:00:00
	2022-08-01 00:00:00
	2022-09-01 00:00:00
	2022-10-01 00:00:00
	2022-11-01 00:00:00
	2022-12-01 00:00:00
	2023-01-01 00:00:00
	2023-02-01 00:00:00
{'origin_airport_id': 11092}: 24 of 364
	2019-01-01 00:00:00
	2019-02-01 00:00:00
	2019-03-01 00:00:00
	2019-04-01 00:00:00
	2019-05-01 00:00:00
	2019-06-01 00:00:00
	2019-07-01 00:00:00
	2019-08-01 00:00:00
	2019-09-01 00:00:00
	2019-10-01 00:00:00
	2019-11-01 00:00:00
	2019-12-01 00:00:00
	2020-01-01 00:00:00
	2020-02-01 00:00:00
	2020-03-01 00:00:00
	2020-04-01 00:00:00
	2020-05-01 00:00:00
	2020-06-01 00:00:00
	2020-07-01 00:00:00
	2020-08-01 00:00:00
	2020-09-01 00:00:00
	2020-10-01 00:00:00
	2020-11-01 00:00:00
	2020-12-01 00:

	2019-04-01 00:00:00
	2019-05-01 00:00:00
	2019-06-01 00:00:00
	2019-07-01 00:00:00
	2019-08-01 00:00:00
	2019-09-01 00:00:00
	2019-10-01 00:00:00
	2019-11-01 00:00:00
	2019-12-01 00:00:00
	2020-01-01 00:00:00
	2020-02-01 00:00:00
	2020-03-01 00:00:00
	2020-04-01 00:00:00
	2020-05-01 00:00:00
	2020-06-01 00:00:00
	2020-07-01 00:00:00
	2020-08-01 00:00:00
	2020-09-01 00:00:00
	2020-10-01 00:00:00
	2020-11-01 00:00:00
	2020-12-01 00:00:00
	2021-01-01 00:00:00
	2021-02-01 00:00:00
	2021-03-01 00:00:00
	2021-04-01 00:00:00
	2021-05-01 00:00:00
	2021-06-01 00:00:00
	2021-07-01 00:00:00
	2021-08-01 00:00:00
	2021-09-01 00:00:00
	2021-10-01 00:00:00
	2021-11-01 00:00:00
	2021-12-01 00:00:00
	2022-01-01 00:00:00
	2022-02-01 00:00:00
	2022-03-01 00:00:00
	2022-04-01 00:00:00
	2022-05-01 00:00:00
	2022-06-01 00:00:00
	2022-07-01 00:00:00
	2022-08-01 00:00:00
	2022-09-01 00:00:00
	2022-10-01 00:00:00
	2022-11-01 00:00:00
	2022-12-01 00:00:00
	2023-01-01 00:00:00
	2023-02-01 00:00:00
{'origin_airp

...

	2020-08-01 00:00:00
	2020-09-01 00:00:00
	2020-10-01 00:00:00
	2020-11-01 00:00:00
	2020-12-01 00:00:00
	2021-01-01 00:00:00
	2021-02-01 00:00:00
	2021-03-01 00:00:00
	2021-04-01 00:00:00
	2021-05-01 00:00:00
	2021-06-01 00:00:00
	2021-07-01 00:00:00
	2021-08-01 00:00:00
	2021-09-01 00:00:00
	2021-10-01 00:00:00
	2021-11-01 00:00:00
	2021-12-01 00:00:00
	2022-01-01 00:00:00
	2022-02-01 00:00:00
	2022-03-01 00:00:00
	2022-04-01 00:00:00
	2022-05-01 00:00:00
	2022-06-01 00:00:00
	2022-07-01 00:00:00
	2022-08-01 00:00:00
	2022-09-01 00:00:00
	2022-10-01 00:00:00
	2022-11-01 00:00:00
	2022-12-01 00:00:00
	2023-01-01 00:00:00
	2023-02-01 00:00:00
{'origin_airport_id': 10409}: 340 of 364
	2019-01-01 00:00:00
	2019-02-01 00:00:00
	2019-03-01 00:00:00
	2019-04-01 00:00:00
	2019-05-01 00:00:00
	2019-06-01 00:00:00
	2019-07-01 00:00:00
	2019-08-01 00:00:00
	2019-09-01 00:00:00
	2019-10-01 00:00:00
	2019-11-01 00:00:00
	2019-12-01 00:00:00
	2020-01-01 00:00:00
	2020-02-01 00:00:00
	2020-03-01 00

	2022-11-01 00:00:00
	2022-12-01 00:00:00
	2023-01-01 00:00:00
	2023-02-01 00:00:00
{'origin_airport_id': 11413}: 347 of 364
	2019-01-01 00:00:00
	2019-02-01 00:00:00
	2019-03-01 00:00:00
	2019-04-01 00:00:00
	2019-05-01 00:00:00
	2019-06-01 00:00:00
	2019-07-01 00:00:00
	2019-08-01 00:00:00
	2019-09-01 00:00:00
	2019-10-01 00:00:00
	2019-11-01 00:00:00
	2019-12-01 00:00:00
	2020-01-01 00:00:00
	2020-02-01 00:00:00
	2020-03-01 00:00:00
	2020-04-01 00:00:00
	2020-05-01 00:00:00
	2020-06-01 00:00:00
	2020-07-01 00:00:00
	2020-08-01 00:00:00
	2020-09-01 00:00:00
	2020-10-01 00:00:00
	2020-11-01 00:00:00
	2020-12-01 00:00:00
	2021-01-01 00:00:00
	2021-02-01 00:00:00
	2021-03-01 00:00:00
	2021-04-01 00:00:00
	2021-05-01 00:00:00
	2021-06-01 00:00:00
	2021-07-01 00:00:00
	2021-08-01 00:00:00
	2021-09-01 00:00:00
	2021-10-01 00:00:00
	2021-11-01 00:00:00
	2021-12-01 00:00:00
	2022-01-01 00:00:00
	2022-02-01 00:00:00
	2022-03-01 00:00:00
	2022-04-01 00:00:00
	2022-05-01 00:00:00
	2022-06-01 00

	2020-10-01 00:00:00
	2020-11-01 00:00:00
	2020-12-01 00:00:00
	2021-01-01 00:00:00
	2021-02-01 00:00:00
	2021-03-01 00:00:00
	2021-04-01 00:00:00
	2021-05-01 00:00:00
	2021-06-01 00:00:00
	2021-07-01 00:00:00
	2021-08-01 00:00:00
	2021-09-01 00:00:00
	2021-10-01 00:00:00
	2021-11-01 00:00:00
	2021-12-01 00:00:00
	2022-01-01 00:00:00
	2022-02-01 00:00:00
	2022-03-01 00:00:00
	2022-04-01 00:00:00
	2022-05-01 00:00:00
	2022-06-01 00:00:00
	2022-07-01 00:00:00
	2022-08-01 00:00:00
	2022-09-01 00:00:00
	2022-10-01 00:00:00
	2022-11-01 00:00:00
	2022-12-01 00:00:00
	2023-01-01 00:00:00
	2023-02-01 00:00:00
{'origin_airport_id': 15454}: 355 of 364
	2019-01-01 00:00:00
	2019-02-01 00:00:00
	2019-03-01 00:00:00
	2019-04-01 00:00:00
	2019-05-01 00:00:00
	2019-06-01 00:00:00
	2019-07-01 00:00:00
	2019-08-01 00:00:00
	2019-09-01 00:00:00
	2019-10-01 00:00:00
	2019-11-01 00:00:00
	2019-12-01 00:00:00
	2020-01-01 00:00:00
	2020-02-01 00:00:00
	2020-03-01 00:00:00
	2020-04-01 00:00:00
	2020-05-01 00

	2023-01-01 00:00:00
	2023-02-01 00:00:00
{'origin_airport_id': 10693}: 362 of 364
	2019-01-01 00:00:00
	2019-02-01 00:00:00
	2019-03-01 00:00:00
	2019-04-01 00:00:00
	2019-05-01 00:00:00
	2019-06-01 00:00:00
	2019-07-01 00:00:00
	2019-08-01 00:00:00
	2019-09-01 00:00:00
	2019-10-01 00:00:00
	2019-11-01 00:00:00
	2019-12-01 00:00:00
	2020-01-01 00:00:00
	2020-02-01 00:00:00
	2020-03-01 00:00:00
	2020-04-01 00:00:00
	2020-05-01 00:00:00
	2020-06-01 00:00:00
	2020-07-01 00:00:00
	2020-08-01 00:00:00
	2020-09-01 00:00:00
	2020-10-01 00:00:00
	2020-11-01 00:00:00
	2020-12-01 00:00:00
	2021-01-01 00:00:00
	2021-02-01 00:00:00
	2021-03-01 00:00:00
	2021-04-01 00:00:00
	2021-05-01 00:00:00
	2021-06-01 00:00:00
	2021-07-01 00:00:00
	2021-08-01 00:00:00
	2021-09-01 00:00:00
	2021-10-01 00:00:00
	2021-11-01 00:00:00
	2021-12-01 00:00:00
	2022-01-01 00:00:00
	2022-02-01 00:00:00
	2022-03-01 00:00:00
	2022-04-01 00:00:00
	2022-05-01 00:00:00
	2022-06-01 00:00:00
	2022-07-01 00:00:00
	2022-08-01 00

MONTH  DAY_OF_MONTH  DAY_OF_WEEK OP_UNIQUE_CARRIER TAIL_NUM  \
0      1             1            2                9E   N931XJ   
1      1             1            2                OH   N723PS   
2      1             1            2                OH   N525EA   
3      1             1            2                OH   N706PS   
4      1             1            2                OH   N262PS   

   OP_CARRIER_FL_NUM  ORIGIN_AIRPORT_ID  DEST_AIRPORT_ID  CRS_DEP_TIME  \
0               3290              10874            10397           600   
1               5495              10874            11057           704   
2               5416              10874            11057          1944   
3               5426              10874            11057          1521   
4               5440              10874            14100           756   

   DEP_TIME  ...  CRS_ELAPSED_TIME ACTUAL_ELAPSED_TIME  DISTANCE  \
0     557.0  ...             129.0               100.0       528   
1     723.0  ...             115.0                82.0       394   
2    1942.0  ...             101.0                96.0       394   
3    1518.0  ...             103.0                93.0       394   
4     800.0  ...              93.0                74.0       335   

   DISTANCE_GROUP  YEAR CARRIER_DELAY  WEATHER_DELAY  NAS_DELAY  \
0               3  2019           NaN            NaN        NaN   
1               2  2019           NaN            NaN        NaN   
2               2  2019           NaN            NaN        NaN   
3               2  2019           NaN            NaN        NaN   
4               2  2019           NaN            NaN        NaN   

   SECURITY_DELAY  LATE_AIRCRAFT_DELAY  
0             NaN                  NaN  
1             NaN                  NaN  
2             NaN                  NaN  
3             NaN                  NaN  
4             NaN                  NaN  

[5 rows x 27 columns]

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [15]:
print(flight_df.shape)

(9251880, 27)


In [16]:
flight_df_expected_shape = (9251880, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`.

In [17]:
flight_df.to_csv('flight.csv')

 # Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.

In [ ]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)